In [17]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_excel('household_dataset.xlsx')
print(df.head())

  Household_ID   Q1   Q2      Q3  Q4   Q5      Q6 Q7   Q8      Q9     Q10
0           H1  Yes  Yes  Medium   4  Yes    High  5  Yes  Medium     Low
1           H2  Yes   No     Low   3   No  Medium  4  Yes     Low  Medium
2           H3   No   No     Low   2  Yes     Low  3   No     Low    High
3           H4  Yes  Yes    High   5  Yes    High  5  Yes    High     Low
4           H5  Yes    –  Medium   4   No  Medium  –  Yes     Low  Medium


In [19]:
# map_binary: function to map binary survey responses (Yes/No) to 0,1
# Positive items: Yes → 1, No → 0 
# Negative items: Yes → 0, No → 1 

def map_binary(value, polarity="positive"):
    if value == "Yes":
        score = 1
    elif value == "No":
        score = 0
    else:
        return None
    
    # Flip if negative polarity
    if polarity == "negative":
        return 1 - score
    return score


In [20]:
# map_ordinal: function to map ordinal survey responses (Low/Medium/High) to 0,1
# Positive items: High → 1, Medium → 0.5, Low → 0 
# Negative items: High → 0, Medium → 0.5, Low → 1 

def map_ordinal(value, polarity="positive"):
    if value == "Low":
        score = 0
    elif value == "Medium":
        score = 0.5
    elif value == "High":
        score = 1
    else:
        return None  # missing or unexpected
    
    # Flip if negative polarity
    if polarity == "negative":
        return 1 - score
    return score

In [21]:
# map_likert: function to map Likert survey responses (1–5 scale) to a normalized 0–1 range.
# Positive items: 1 → 0, 2 → 0.25, 3 → 0.5, 4 → 0.75, 5 → 1
# Negative items: 1 → 1, 2 → 0.75, 3 → 0.5, 4 → 0.25, 5 → 0'''

def map_likert(value, polarity="positive"):
    try:
        # Convert to integer in case Excel stores it as text
        val = int(value)
        # Base mapping: (x - 1) / 4 → 1→0, 3→0.5, 5→1
        score = (val - 1) / 4
    except:
        return None  # If blank or invalid, mark as missing
    
    # Flip if negative polarity
    if polarity == "negative":
        return 1 - score
    return score


In [22]:
# Infrastructure indicator with missing value handling
def compute_infrastructure(row):
    scores = row[['Q1_score', 'Q2_score', 'Q3_score', 'Q4_score', 'Q5_score']]
    answered = scores.count()  # number of non-missing answers
    if answered >= 3:           # require at least 3 of 5 answered
        return scores.mean()
    else:
        return np.nan           # mark as missing if too few answers

# Socioeconomic indicator with missing value handling
def compute_socioeconomic(row):
    scores = row[['Q6_score', 'Q7_score', 'Q8_score', 'Q9_score', 'Q10_score']]
    answered = scores.count()
    if answered >= 3:           # require at least 3 of 5 answered
        return scores.mean()
    else:
        return np.nan

# Composite index: only compute if both indicators are present
def compute_composite(row):
    if pd.notna(row['Infrastructure']) and pd.notna(row['Socioeconomic']):
        return (row['Infrastructure'] + row['Socioeconomic']) / 2
    else:
        return np.nan

In [23]:
df = df.replace(["", "NA", "None","-"], pd.NA)
# Apply to columns
df['Q1_score'] = df['Q1'].apply(lambda x: map_binary(x, polarity="positive"))  
df['Q2_score'] = df['Q2'].apply(lambda x: map_binary(x, polarity="positive"))  
df['Q5_score'] = df['Q5'].apply(lambda x: map_binary(x, polarity="positive"))  
df['Q8_score'] = df['Q8'].apply(lambda x: map_binary(x, polarity="positive"))  
# Positive ordinal questions
df['Q3_score'] = df['Q3'].apply(lambda x: map_ordinal(x, polarity="positive"))
df['Q6_score'] = df['Q6'].apply(lambda x: map_ordinal(x, polarity="positive"))
df['Q9_score'] = df['Q9'].apply(lambda x: map_ordinal(x, polarity="positive"))

# Negative ordinal question (Debt burden, Q10)
df['Q10_score'] = df['Q10'].apply(lambda x: map_ordinal(x, polarity="negative"))

# Positive Likert questions
df['Q4_score'] = df['Q4'].apply(lambda x: map_likert(x, polarity="positive"))
df['Q7_score'] = df['Q7'].apply(lambda x: map_likert(x, polarity="positive"))

print(df[['Q1_score','Q2_score','Q3_score','Q4_score','Q5_score','Q6_score','Q7_score','Q8_score','Q9_score','Q10_score']].head())

   Q1_score  Q2_score  Q3_score  Q4_score  Q5_score  Q6_score  Q7_score  \
0         1       1.0       0.5      0.75         1       1.0      1.00   
1         1       0.0       0.0      0.50         0       0.5      0.75   
2         0       0.0       0.0      0.25         1       0.0      0.50   
3         1       1.0       1.0      1.00         1       1.0      1.00   
4         1       NaN       0.5      0.75         0       0.5       NaN   

   Q8_score  Q9_score  Q10_score  
0         1       0.5        1.0  
1         1       0.0        0.5  
2         0       0.0        0.0  
3         1       1.0        1.0  
4         1       0.0        0.5  


In [24]:
df['Infrastructure'] = df.apply(compute_infrastructure, axis=1)
df['Socioeconomic'] = df.apply(compute_socioeconomic, axis=1)
df['Composite_Index'] = df.apply(compute_composite, axis=1)

# Check results
print(df[['Infrastructure', 'Socioeconomic', 'Composite_Index']].head())

   Infrastructure  Socioeconomic  Composite_Index
0          0.8500           0.90          0.87500
1          0.3000           0.55          0.42500
2          0.2500           0.10          0.17500
3          1.0000           1.00          1.00000
4          0.5625           0.50          0.53125


In [26]:

ordered_columns = ['Q1', 'Q1_score','Q2', 'Q2_score','Q3', 'Q3_score','Q4', 'Q4_score',
    'Q5', 'Q5_score','Q6', 'Q6_score','Q7', 'Q7_score','Q8', 'Q8_score', 'Q9', 'Q9_score','Q10', 'Q10_score',
    'Infrastructure','Socioeconomic','Composite_Index']

df = df[ordered_columns]
df.to_excel('survey_results.xlsx', index=False)
print("Data exported successfully!")

Data exported successfully!


In [27]:
correlation_matrix = df[['Infrastructure', 'Socioeconomic', 'Composite_Index']].corr()
print("\nCorrelation matrix:") 
print(correlation_matrix) 


Correlation matrix:
                 Infrastructure  Socioeconomic  Composite_Index
Infrastructure         1.000000       0.953592         0.988819
Socioeconomic          0.953592       1.000000         0.987831
Composite_Index        0.988819       0.987831         1.000000


In [29]:
# Top 5 households by Composite_Index
top5 = df.sort_values('Composite_Index', ascending=False).head(5)
print("\nTop 5 households by Composite_Index:")
print(top5[['Composite_Index']])

# Bottom 5 households by Composite_Index
bottom5 = df.sort_values('Composite_Index', ascending=True).head(5)
print("\nBottom 5 households by Composite_Index:")
print(bottom5[['Composite_Index']])



Top 5 households by Composite_Index:
    Composite_Index
3               1.0
6               1.0
15              1.0
24              1.0
21              1.0

Bottom 5 households by Composite_Index:
    Composite_Index
8             0.025
17            0.025
14            0.050
5             0.050
23            0.050


In [ ]:

# Summary statistics of all the responses as well as indicators and composite index 
cols = ['Q1_score','Q2_score','Q3_score','Q4_score','Q5_score', 'Q6_score','Q7_score','Q8_score','Q9_score','Q10_score', 'Infrastructure','Socioeconomic','Composite_Index'] # Compute mean, min, max 
summary = pd.DataFrame({ 'mean': df[cols].mean(), 'median': df[cols].median(), 'min': df[cols].min(), 'max': df[cols].max() })
print(summary)

                     mean   median    min  max
Q1_score         0.680000  1.00000  0.000  1.0
Q2_score         0.478261  0.00000  0.000  1.0
Q3_score         0.480000  0.50000  0.000  1.0
Q4_score         0.610000  0.75000  0.000  1.0
Q5_score         0.480000  0.00000  0.000  1.0
Q6_score         0.520000  0.50000  0.000  1.0
Q7_score         0.706522  0.75000  0.250  1.0
Q8_score         0.680000  1.00000  0.000  1.0
Q9_score         0.440000  0.50000  0.000  1.0
Q10_score        0.520000  0.50000  0.000  1.0
Infrastructure   0.547000  0.56250  0.000  1.0
Socioeconomic    0.570000  0.65000  0.050  1.0
Composite_Index  0.558500  0.53125  0.025  1.0
